In [22]:
import pandas as pd

## Reorganizing Data into a "Tidy" / "Long" / "Tabular" / "Design" DataFrame
### `pd.melt()`

The `pd.melt()` function and `DataFrame.melt()` method take a single dataframe and make it **taller** by taking data stored in column names and putting it into the rows along with the rest of the data, adding extra metadata in the process.

For example, it can turn this `df` DataFrame:

| Month | Year | Monday | Tuesday | Wednesday |
| :--:  | :--: | :--:   | :--:    | :--:      |
| January | 2021 | 0 | -2 | -1 |
| February | 2021 | 2 | 4 | -2 |

into this:

| Month | Year | Weekday | Temperature |
| :--:  | :--: | :--:    |  :--:       |
| January | 2021 | Monday | 0 |
| January | 2021 | Tuesday | -2 |
| January | 2021 | Wednesday | -1 |
| February | 2021 | Monday | 2 |
| February | 2021 | Tuesday | 4 | 
| February | 2021 | Wednesday | -2 |

with one line of code:

```python
pd.melt(
    df, 
    id_vars=['Month', 'Year'],  # The columns that should stay the same
    value_vars=['Monday', 'Tuesday', 'Wednesday'],   # The columns that should melt
    var_name='Weekday',  # The new Column that will represent the melted column name's variable
    value_name='Temperature'  # The new Column that the data represents
)
```

**Note**: Melting a dataframe also called *"tidying"* data, making a *"long"* dataframe from a *"wide"* dataframe, or building a *design matrix*

#### Exercises

Let's practice tidying dataframes with the `pd.melt()` function.  

In [23]:
df = pd.DataFrame({
    'Attendee': ['Mark', 'Susan', 'June', 'Lingling'],
    'Group': ['MPI', 'LMU', 'LMU', 'MPI'],
    'Monday': [True, True, False, True],
    'Tuesday': [False, True, False, True],
    'Wednesday': [True, True, True, True],
})
df

,Attendee,Group,Monday,Tuesday,Wednesday
0,Mark,MPI,True,False,True
1,Susan,LMU,True,True,True
2,June,LMU,False,False,True
3,Lingling,MPI,True,True,True


Melt this dataset into four columns: "Attendee", "Group", and "DayOfWeek", "Attended"

Melt the 1948 U.S. Unemployment dataset into 3 columns: Year, Month, and Unemployment

In [19]:
from bokeh.sampledata import unemployment1948
data = unemployment1948.data[['Year', 'Jan', 'Feb', 'Mar']]
data.head()

,Year,Jan,Feb,Mar
0,1948,4.0,4.7,4.5
1,1949,5.0,5.8,5.6
2,1950,7.6,7.9,7.1
3,1951,4.4,4.2,3.8
4,1952,3.7,3.8,3.3


Melt the Gapminder Fertility Rate dataset into 3 columns: Country, Year, and FertilityRate

In [2]:
import bokeh
# bokeh.sampledata.download()
from bokeh.sampledata import gapminder
gapminder.fertility.head()

,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,...,7.136,6.930,6.702,6.456,6.196,5.928,5.659,5.395,5.141,4.900
Albania,5.711,5.594,5.483,5.376,5.268,5.160,5.050,4.933,4.809,4.677,...,2.004,1.919,1.849,1.796,1.761,1.744,1.741,1.748,1.760,1.771
Algeria,7.653,7.655,7.657,7.658,7.657,7.652,7.641,7.622,7.591,7.548,...,2.448,2.507,2.580,2.656,2.725,2.781,2.817,2.829,2.820,2.795
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Merging Exercise

Tidy all four datasets in the gapminder data and merge them together into a single tidy table!

In [30]:
fertility = gapminder.fertility.reset_index().melt(
    id_vars=['Country'],
    value_vars=gapminder.fertility.columns,
    var_name='Year',
    value_name='FertilityRate',    
)

life = gapminder.life_expectancy.reset_index().melt(
    id_vars=['Country'],
    value_vars=gapminder.life_expectancy.columns,
    var_name='Year',
    value_name='LifeExpectancy',    
)

pop = gapminder.population.reset_index().melt(
    id_vars=['Country'],
    value_vars=gapminder.population.columns,
    var_name='Year',
    value_name='Population',    
)

regions = gapminder.regions.reset_index()

pop.head()

,Country,Year,Population
0,Afghanistan,1964,10474903.0
1,Albania,1964,1817098.0
2,Algeria,1964,11654905.0
3,American Samoa,1964,22672.0
4,Andorra,1964,17438.0


In [45]:
dfl = fertility.merge(
    life, 
    left_on=('Country', 'Year'), 
    right_on=('Country', 'Year')
).merge(
    pop, 
    left_on=('Country', 'Year'), 
    right_on=('Country', 'Year')    
).merge(
    regions,
    left_on='Country',
    right_on='Country',
)
dfl.to_parquet('gapminder.parquet')
dfl.head()
dfl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12200 entries, 0 to 12199
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         12200 non-null  object 
 1   Year            12200 non-null  object 
 2   FertilityRate   10100 non-null  float64
 3   LifeExpectancy  10111 non-null  float64
 4   Population      11806 non-null  float64
 5   Group           12200 non-null  object 
 6   ID              12150 non-null  object 
dtypes: float64(3), object(4)
memory usage: 762.5+ KB


In [46]:
!dir 

 Volume in drive C is Windows
 Volume Serial Number is 3605-60C0

 Directory of C:\Users\nickdg\Projects\DataScienceTuts

06/08/2022  05:31 PM    <DIR>          .
06/08/2022  05:31 PM    <DIR>          ..
06/08/2022  03:16 PM    <DIR>          .ipynb_checkpoints
06/08/2022  08:28 AM            57,000 1_AnalyzingDataFrames.ipynb
06/08/2022  03:18 PM            47,417 4_ReorganizingDataFrames.ipynb
06/08/2022  09:20 AM           112,648 aa.h5
06/07/2022  08:36 AM            65,492 CleaningCategoricalTables.ipynb
06/06/2022  06:32 PM    <DIR>          data
06/08/2022  12:25 PM           200,704 data.db
06/07/2022  07:33 AM           874,276 E.coli MG1655 selection glpD protein BLAST.xlsx
06/08/2022  12:12 PM           959,764 gap.h5
06/08/2022  11:57 AM                 0 gap.xlsx
06/08/2022  12:00 PM                 0 gap2.xlsx
06/08/2022  12:13 PM         3,046,687 gap3.h5
06/08/2022  01:25 PM           352,256 gapminder.db
06/08/2022  05:31 PM           270,506 gapminder.parquet
06/08/2

In [43]:
df = pd.DataFrame()
df['Fertility'] = fertility.set_index(['Country', 'Year'])['FertilityRate']
df['Life'] = life.set_index(['Country', 'Year'])['LifeExpectancy']

df

,,Fertility,Life
Country,Year,,
Afghanistan,1964,7.671,33.639
Albania,1964,5.711,65.475
Algeria,1964,7.653,47.953
American Samoa,1964,NaN,NaN
Andorra,1964,NaN,NaN
...,...,...,...
Virgin Islands (U.S.),2013,2.487,80.152
"Yemen, Rep.",2013,4.075,63.112
Zambia,2013,5.687,58.105
